# Regression in PyTorch

In [1]:
import pandas as pd # pandas for reading in the csv data and visualizing it
import torch # PyTorch for building and training the network
import torch.nn as nn # For building the model architecture
from sklearn.model_selection import train_test_split # for splitting the dataset into train and test sets

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Simple Linear Regression

### Reading in the data

In [2]:
# dataset path
# read in the dataset

### Training the model

### Evaluating model performance

## Multi Regression - Nutrients Data

### Reading in the data

In [3]:
# read in the data
nutrients_dataframe = pd.read_csv('../Data/nutrients.csv')

# print first 5 rows
nutrients_dataframe.head(5)

,Name,Energy,Protein,Fat,Calcium,Iron
0,Braised beef,340,20,28,9,2.6
1,Hamburger,245,21,17,9,2.7
2,Roast beef,420,15,39,7,2.0
3,Beefsteak,375,19,32,9,2.6
4,Canned beef,180,22,10,17,3.7


### Preparing the data as input

We are training a neural network to predict the energy of a food (basically calories) based on the other contents of the food. We don't want to use the food name since that would not be helpful for predicting the energy of food that is not in our dataset.

In [4]:
# train_data_input, train_data_
X = nutrients_dataframe[['Protein', 'Fat', 'Calcium', 'Iron']].values
y = nutrients_dataframe[['Energy']].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

### Defining the model architecture

In [5]:
# class MultiRegressionModel(nn.Module):
# 	def __init__(self):
# 		super(MultiRegressionModel, self).__init__()
# 		# define fully connected layers
# 		self.fc1 = nn.Linear(4, 10) # 4 input features -> 10 neurons
# 		self.fc2 = nn.Linear(10, 1) # 10 neurons -> 1 output neuron (energy)
	
# 	def forward(self, x):
# 		x = self.fc1(x)
# 		x = self.fc2(x)
# 		return x

In [6]:
class MultiRegressionModel(nn.Module):
	def __init__(self):
		super(MultiRegressionModel, self).__init__()
		# define fully connected layers
		self.fc1 = nn.Linear(4, 10) # 4 input features -> 10 neurons
		self.fc2 = nn.Linear(10, 10)
		self.fc3 = nn.Linear(10, 10)
		self.fc4 = nn.Linear(10, 10)
		self.fc5 = nn.Linear(10, 1) # 10 neurons -> 1 output neuron (energy)
	
	def forward(self, x):
		x = self.fc1(x)
		x = self.fc2(x)
		x = self.fc3(x)
		x = self.fc4(x)
		x = self.fc5(x)
		return x

### Training the model

In [7]:
# create an instance of the model
model = MultiRegressionModel()

# choose mean squared error as the loss function
criterion = nn.MSELoss()

# choose Adam to optimize the neural net weights
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
EPOCHS = 1000
for epoch in range(EPOCHS):
	model.train()
	optimizer.zero_grad()
	predictions = model(X_train)
	loss = criterion(predictions, y_train)
	loss.backward()
	optimizer.step()

	if (epoch+1) % 100 == 0:
		print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 53.5867
Epoch [200/1000], Loss: 45.9571
Epoch [300/1000], Loss: 43.8927
Epoch [400/1000], Loss: 41.9313
Epoch [500/1000], Loss: 40.1601
Epoch [600/1000], Loss: 38.6627
Epoch [700/1000], Loss: 37.4705
Epoch [800/1000], Loss: 36.5761
Epoch [900/1000], Loss: 35.9450
Epoch [1000/1000], Loss: 35.5274


### Evaluate the model

In [8]:
model.eval()
with torch.no_grad():
	predictions = model(X_test)
	test_loss = criterion(predictions, y_test)
	print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 25.5372


In [9]:
# Predicting using the trained model
new_data = torch.tensor([[25, 35, 12, 3.0]], dtype=torch.float32)
predicted_energy = model(new_data)
print(f'Predicted Energy: {predicted_energy.item():.4f}')

Predicted Energy: 421.2410
